In [2]:
import json

In [ ]:
# Split symptoms into a list
def extract_symptoms(string):
    # Splitting on commas
    split_comma_strings = [symptom.strip() for symptom in string.split(',')]

    # Want to handle any "and" that separates symptoms.
    # Ex. 'Headache and fever'
    # Ex. 'Headache'
    # Headache should count as a common symptom for both diseases
    split_and_symptoms = []
    for symp in split_comma_strings:
        symp_part = [symptom.strip() for symptom in string.split(' and ')]
        split_and_symptoms.append(symp_part)


    # Then just removing any trailing or leading punctuation if there is any.
    # Probably not but just making sure since its a large dataset
    strip_symptoms = []
    for symptom in split_and_symptoms:
        new = symptom.strip('.,')
        strip_symptoms.append(new)

    # Also normalizing the string to just lowercase for consistency
    symptoms = []
    for symptom in strip_symptoms:
        new = symptom.lower()
        symptoms.append(new)

    return symptoms

In [ ]:
# Loading the json dataset and removing the treatment field since it isn't needed right now.
# Gotta iterate through each field and make a new dictionary

def remove_treatment_field():
    new_dataset = []

    try:

        with open('disease_database_en.json', 'r', encoding='utf-8') as fd:
            data = json.load(fd)

            for entry in data:
                new_entry = {
                    "disease_id": entry["disease_id"],
                    "disease": entry["disease"],
                    "symptoms": entry["common_symptom"]
                }
                new_dataset.append(new_entry)

        print(new_dataset)

        with open('new_dataset.json', 'w', encoding='utf-8') as output:
            json.dump(new_dataset, output, indent = 2)

        print("Saved dataset!")
        return new_dataset

    except Exception as error:
        print(f"An error occured: {error}")

remove_treatment_field()